In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd

In [3]:
import lib
import plotting
import markov_chain as mc
import markov_tools as mt

In [4]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison

In [5]:
from configparser import ConfigParser
cfg = ConfigParser()
cfg.read("config.cfg")

['config.cfg']

In [6]:
from tqdm._tqdm_notebook import tqdm_notebook as tqdm # we manually import the notebook submodule as the normal one struggles with jupyterlab
tqdm.pandas() # this enables us to use progress_apply instead of apply

In [7]:
results_prefix = "[WIKIDATA]"

sequence_file = os.path.join(cfg.get("directory", "exchange"), results_prefix + "001_clustering.p")
labels_file = os.path.join(cfg.get("directory", "exchange"), "[wikidata]004_labels.p")

In [8]:
df = pd.read_pickle(sequence_file)
df.head()

,length,sequence,user_id,stat_dist,kmeans_2,kmeans_3,kmeans_4,kmeans_5,kmeans_6,kmeans_7,kmeans_8,kmeans_9,kmeans_10
0,1511,"[REVERT_MANUAL, BREAK, CLAIM_CREATE, BREAK, CL...",1,"[0.022260285362644863, 0.01999793118304165, 0....",1,0,3,2,2,4,1,2,2
1,9,"[ENTITY_CREATE, BREAK, SITELINK_ADD, BREAK, ME...",1000036,"[0.020070153072094386, 0.02007015307209438, 0....",0,0,2,4,0,0,0,0,0
2,7,"[SITELINK_ADD, BREAK, SITELINK_ADD, BREAK, SIT...",1000078,"[0.01947397679104996, 0.019473976791049966, 0....",0,2,1,1,1,1,5,3,3
3,9,"[ENTITY_CREATE, BREAK, SITELINK_REMOVE, SITELI...",100008,"[0.02042483183496783, 0.020424831834967825, 0....",0,2,1,1,1,1,5,0,0
4,44,"[SITELINK_ADD, BREAK, SITELINK_ADD, BREAK, SIT...",100012,"[0.01973684210526316, 0.019736842105263157, 0....",0,2,1,1,1,1,5,3,3


In [9]:
labels = pd.read_pickle(labels_file)['label'].values

We test only one clustering (numer of clusters) at a time.  
For each label/action we conduct a pairwise tukey's test to highlight the actions that show significant difference.

Note: as this is a pairwise comparison for each single label this generates a lot of output!

In [10]:
num_clusters = 8
df_selector = "kmeans_{n}".format(n=num_clusters)

In [13]:
for i, l in enumerate(labels):
    vals = df['stat_dist'].apply(lambda x: x[i])
    lbls = df[df_selector]
    comp_object = MultiComparison(vals, lbls)
    hsd = comp_object.tukeyhsd(alpha=0.01)
    print("{l}: {s} / 28".format(l=l, s=sum(hsd.reject)))

ALIAS_ADD: 19 / 28
ALIAS_REMOVE: 12 / 28
ALIAS_UPDATE: 14 / 28
BREAK: 28 / 28
CLAIM_CREATE: 18 / 28
CLAIM_REMOVE: 17 / 28
CLAIM_UPDATE: 18 / 28
CLAIM_UPDATEVALUE: 11 / 28
CREATE_MANUAL: 9 / 28
DESCRIPTION_ADD: 22 / 28
DESCRIPTION_REMOVE: 13 / 28
DESCRIPTION_UPDATE: 24 / 28
ENTITY_CREATE: 18 / 28
ENTITY_OVERRIDE: 13 / 28
ENTITY_REDIRECT: 19 / 28
ENTITY_UPDATE: 13 / 28
FORM_ADD: 13 / 28
FORM_REMOVE: 13 / 28
FORM_UPDATE: 13 / 28
LABEL_ADD: 20 / 28
LABEL_REMOVE: 7 / 28
LABEL_UPDATE: 14 / 28
LEXEME_CREATE: 13 / 28
LEXEME_UPDATE: 13 / 28
MERGE: 23 / 28
MOVE_MANUAL: 13 / 28
PROPERTY_CREATE: 13 / 28
PROTECT_MANUAL: 13 / 28
QUALIFIER_ADD: 13 / 28
QUALIFIER_REMOVE: 13 / 28
QUALIFIER_UPDATE: 13 / 28
REFERENCE_ADD: 13 / 28
REFERENCE_REMOVE: 13 / 28
REFERENCE_UPDATE: 13 / 28
REMOVE_MANUAL: 12 / 28
RENAME_MANUAL: 12 / 28
REVERT: 12 / 28
REVERT_MANUAL: 10 / 28
SENSE_ADD: 13 / 28
SENSE_REMOVE: 13 / 28
SENSE_UPDATE: 13 / 28
SITELINK_ADD: 22 / 28
SITELINK_REMOVE: 17 / 28
SITELINK_UPDATE: 17 / 28
